In [1]:
%matplotlib inline
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from mpl_toolkits.basemap import Basemap
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, HTML
import folium
from folium.plugins import HeatMap

In [37]:
data = pd.read_csv('taxi_7_data.csv', sep=';', index_col='dt')
data.index = pd.to_datetime(data.index)

In [38]:
regions = pd.read_csv('taxi_7_regions.csv', sep=';', index_col=False)

# Неделя 7. Оформление проекта в виде интерактивного демо.

Итоговые временные ряды были рассчитаны с помощью рекуррентной нейронной сети по методике, изложенной здесь - http://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

В этой демонстрации используются данные за май (реальные и прогнозные). Поскольку прогнозирование занимает довольно длительное время, то прогнозные данные были рассчитаны заранее и сохранены в датасет.

В соответствии с требованиями демо состоит из двух частей.

## 1. Карты с визуализацией реального и прогнозируемого спроса на такси в выбираемый пользователем момент времени (для отображения используется тепловая карта)

In [53]:
lon0 = -73.9778 
lat0 = 40.705825

def DateForFilter(day, hour):
    res = '2016-05-{0:0>2} {1:0>2}:00:00'.format(day, hour)
    return res

def DateForView(day, hour):
    res = u'Дата: {0:0>2}.05.2016 Время: {1:0>2}:00:00'.format(day, hour)
    return res

def VisualizeDataForDT(day, hour):
    dateFilter = DateForFilter(day, hour)
    data_filter = data[data.index==dateFilter]
    data_filter_arr = data_filter.as_matrix()[0]
    data_real = data_filter_arr[0:102]
    data_forecast = data_filter_arr[102:204]
                       
    map_real = folium.Map(location=[lat0, lon0], zoom_start=10)
    map_real.add_child(HeatMap(zip(regions['lat'].values, regions['lon'].values, data_real), radius = 10))    
    map_real.save('real.html')

    map_forecast = folium.Map(location=[lat0, lon0], zoom_start=10)
    map_forecast.add_child(HeatMap(zip(regions['lat'].values, regions['lon'].values, data_forecast), radius = 10))    
    map_forecast.save('forecast.html')    
    
    real_title = u'Реальные данные:'
    forecast_title = u'Прогноз:'
    
    htmlText = ''
    htmlText = htmlText + '<table>'
    htmlText = htmlText + '<tr><td colspan="2" style="text-align:center;"><h3>' + DateForView(day, hour) + '</h3></td></tr>'
    htmlText = htmlText + '<tr><td><h3 style="text-align:center;">' + real_title + '</h3></td><td><h3 style="text-align:center;">' + forecast_title + '</h3></td></tr>'
    htmlText = htmlText + '<tr><td><iframe src="real.html" width="400" height="300"></iframe></td>'
    htmlText = htmlText + '<td><iframe src="forecast.html" width="400" height="300"></iframe></td></tr>'
    htmlText = htmlText + '</table>'
   
    display(HTML(htmlText))
    
WData = interactive(VisualizeDataForDT, day=widgets.IntSlider(min=1,max=31,step=1,value=1,description=u'День:'), hour=widgets.IntSlider(min=0,max=23,step=1,value=0,description=u'Час:'))
display(WData)    

## 2. Временной ряд фактического и прогнозируемого спроса на такси в выбираемой области

К сожалению не удалось для используемой библиотеки Folium прикрепить перерисовку временного ряда к событию клика на маркер. Поэтому отдельно выведем карту с маркерами регионов (маркер выводится в центре региона) и отдельно по номеру выбираемой области будем отрисовывать временные ряды.

In [45]:
map_marker = folium.Map(location=[lat0+0.07, lon0], zoom_start=11)
  
for reg, lon, lat in zip(regions['region'], regions['lon'], regions['lat']):
    text = u'Регион №' + str(reg)
    folium.Marker([lat, lon], popup=text).add_to(map_marker)    
    
map_marker    

In [51]:
reg_list = [int(r) for r in regions['region'].as_matrix()]

def PlotForecast(current_region):
    current_region_col = 'region'+str(current_region)
    current_region_col_pred = 'pred_' + current_region_col 
    
    #Данные для построения графика
    x_data = range(len(data[current_region_col]))
    x_labels = data.index.tolist()
    y_data = data[current_region_col]
    y_forecast = data[current_region_col_pred]
    #Метки для построения графика
    ticks = range(0, len(x_data), 12)
    x_data_ticks = [x_data[i] for i in ticks] + [x_data[len(x_data)-1]]
    x_labels_ticks = [x_labels[i] for i in ticks] + [x_labels[len(x_labels)-1]]
    f, (ax1) = plt.subplots(figsize=(20,10))
    ax1.set(title='Taxi rides in May 2016', xlabel='Date and hour', ylabel='Count of rides')
    pl_real = ax1.plot(x_data, y_data, label='Real data')
    pl_forecast = ax1.plot(x_data, y_forecast, 'g', label='Forecast')
    plt.xticks(x_data_ticks, x_labels_ticks, rotation='vertical')
    handles, labels = ax1.get_legend_handles_labels()
    ax1.legend(handles, labels)
    plt.show()
    
def VisualizeForecast(region):
    if len(region)>0:
        r = region[0]
        PlotForecast(r)

WForecast = interactive(VisualizeForecast, region=widgets.SelectMultiple(options=reg_list, description=u'Выберите регион:', disabled=False))
display(WForecast)   

Можно отметить, что рекуррентная нейронная сеть хорошо повторяет форму сигнала, но не всегда точно прогнозирует амплитуду. 